## Prediction Model

**Creating a predictive analysis to determine company's loan decisions that were offered and rejected by the Lending Club**





In [1]:
# First import librar dependacies : 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [4]:
# Run Boto3 to establish data connection to s3 bucket
import boto3
import awsKeyConfig
from io import StringIO

s3 = boto3.client('s3',
aws_access_key_id=awsKeyConfig.keyID,
aws_secret_access_key=awsKeyConfig.secretKey,
region_name='us-east-1'
)

In [6]:
# Retrieves the Bucket Name and Key <file_name> and 
obj = s3.get_object(Bucket='davis-data-cloud-of-wonders', Key='joined_loan_output_data.csv')
data = obj['Body'].read().decode('utf-8')

# Validates whether the connection to s3 is successfull or fail 
status = obj.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 put_object response. Status - {status}")
else:
    print(f"Unsuccessful S3 put_object response. Status - {status}")


# Use StringIO to convert the string data to a file-like object
data_file = StringIO(data)

# Create a DataFrame from the CSV data
loan_df = pd.read_csv(data_file)

loan_df.head()

Successful S3 put_object response. Status - 200


,loan_amnt_x,term,int_rate,installment,grade,sub_grade,emp_title,emp_length_x,home_ownership,annual_inc,...,fico_score_x,loan_app_id,loan_amnt_y,issue_d_y,purpose_y,fico_score_y,dti_y,state,emp_length_y,policy_code
0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,...,677.0,1,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,0.1000,NM,4 years,0.0
1,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,...,717.0,2,1000.0,2007-05-26,Consolidating Debt,703.0,0.1000,MA,< 1 year,0.0
2,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,...,697.0,3,11000.0,2007-05-27,Want to consolidate my debt,715.0,0.1000,MD,1 year,0.0
3,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,...,787.0,4,6000.0,2007-05-27,waksman,698.0,0.3864,MA,< 1 year,0.0
4,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,...,697.0,5,1500.0,2007-05-27,mdrigo,509.0,0.0943,MD,< 1 year,0.0


In [9]:
# Review the datatypes and columns dataset
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(loan_df.dtypes)

loan_amnt_x            float64
term                    object
int_rate               float64
installment            float64
grade                   object
sub_grade               object
emp_title               object
emp_length_x            object
home_ownership          object
annual_inc             float64
verification_status     object
issue_d_x               object
loan_status             object
purpose_x               object
addr_state              object
dti_x                  float64
fico_score_x           float64
loan_app_id              int64
loan_amnt_y            float64
issue_d_y               object
purpose_y               object
fico_score_y           float64
dti_y                  float64
state                   object
emp_length_y            object
policy_code            float64
dtype: object


In [10]:
# Display the number of unique values in each column
for a in loan_df:
    print(a, len(loan_df[a].unique()))

loan_amnt_x 1573
term 3
int_rate 674
installment 93302
grade 8
sub_grade 36
emp_title 512695
emp_length_x 12
home_ownership 7
annual_inc 89369
verification_status 4
issue_d_x 140
loan_status 10
purpose_x 15
addr_state 52
dti_x 10846
fico_score_x 49
loan_app_id 2260701
loan_amnt_y 2158
issue_d_y 2177
purpose_y 61365
fico_score_y 561
dti_y 39796
state 52
emp_length_y 12
policy_code 2
